In [1]:
from pruning_def import *
options.ii_on=True

from exh.model import options
options.dom_quant=3

from IPython.core.display import display, HTML
# from exh.utils import jprint # for fancy displays in Jupyter Notebook

In [2]:
P=Pred(name="P")
P.depends(1)
Q=Pred(name="Q")
Q.depends(1)
R=Pred(name="R")
R.depends(1)

# Relevance

We'd like to know given any set of relevant propositions what all the propositions it obligatorily renders relevant are.

Background assumption: Relevance is closed under Boolean operations (conjunction and negation).

The function Cells finds all the cells in the partition induced by a set of relevant alternatives. The function Rel finds all the unions of cells in this partition (i.e., all the propositions made obligatorily relevant). 

In [3]:
Alt=[a|b,a,b,a&b]
univ=Universe(fs=Alt)
relevant=Alt.copy()
jprint("Set of alternatives of {0}: ".format(a|b),Alt)
jprint("Cells in partition induced by this set of alternatives: ",Cells(Alt))
jprint("Unions of cells in partition induced by this set of alternatives (relevant propositions): ",Rel(Alt))

Of course with more alternatives the set of relevant alternatives becomes bigger.
Case in point: disjunction in the scope of a universal quantifier.

In [4]:
prej_universal=Ax>P|Q
exh_universal=Exh(prej_universal,ii=True)
jprint("Set of alternatives of {0}: ".format(prej_universal),exh_universal.alts)
jprint("Cells in partition induced by this set of alternatives: ",Cells(exh_universal.alts))
rel_exh_universal=Rel(exh_universal.alts)
jprint("Number of unions of cells in partition induced by this set of alternatives (relevant propositions): ",len(rel_exh_universal))

# Pruning

Fox and Katzir 2011: Constraining pruning by relevance allows for undesired pruning choices.

*Pruning by Relevance*: 
For any $C \subseteq ALT$: Exh(p,C) is only licensed if every member of $ALT \setminus C$ isn't made obligatorily relevant by $C$.

In [5]:
jprint("Is subset {0} a licit choice of pruning from {1} given Pruning by Relevance?".format([a|b,a,b],Alt),PrunRelevance(a|b,Alt,[a|b,a,b]))
jprint("Is subset {0} a licit choice of pruning from {1} given Pruning by Relevance?".format([a|b,a],Alt),PrunRelevance(a|b,Alt,[a|b,a]))
jprint("Is subset {0} a licit choice of pruning from {1} given Pruning by Relevance?".format([a|b],Alt),PrunRelevance(a|b,Alt,[a|b]))

We'd like to block the second pruning choice above where $b$ is pruned but $a$ is not, because allowing it would predict "a or b" to be able to mean "b and not a".
Fox and Katzir suggest the constraint of Exhaustive Relevance.

*Pruning by Exhaustive Relevance*: 

For any $C \subseteq ALT$: Exh(p,C) is only licensed if for every $A \in ALT \setminus C$: Exh(A,C) isn't made obligatorily relevant by $C$.

In [6]:
jprint("Is subset {0} a licit choice of pruning from {1} given Pruning by Exhaustive Relevance?".format([a|b,a,b],Alt),PrunExhaustiveRelevance(a|b,Alt,[a|b,a,b]))
jprint("Is subset {0} a licit choice of pruning from {1} given Pruning by Exhaustive Relevance?".format([a|b,a],Alt),PrunExhaustiveRelevance(a|b,Alt,[a|b,a]))
jprint("Is subset {0} a licit choice of pruning from {1} given Pruning by Exhaustive Relevance?".format([a|b],Alt),PrunExhaustiveRelevance(a|b,Alt,[a|b]))

One may consider other constraints on pruning which would give the same results in this case. We suggest the following:
    
*Pruning by Non-Settling* (or: don't judge anyone in their absence): 

For any $C \subseteq ALT$: $Exh(p,C)$ is only licensed if for every $A \in ALT \setminus C$: $Exh(p,C) \not\Rightarrow A$ and $Exh(p,C) \not\Rightarrow \neg A$.

In [7]:
jprint("Is subset {0} a licit choice of pruning from {1} given Pruning by Non-Settling?".format([a|b,a,b],Alt),PrunSettling(a|b,Alt,[a|b,a,b]))
jprint("Is subset {0} a licit choice of pruning from {1} given Pruning by Non-Settling?".format([a|b,a],Alt),PrunSettling(a|b,Alt,[a|b,a]))
jprint("Is subset {0} a licit choice of pruning from {1} given Pruning by Non-Settling?".format([a|b],Alt),PrunSettling(a|b,Alt,[a|b]))

More generally, it would be helpful to know, for any prejacent and set of alternatives, what choices of pruning are possible (and as a result what meanings a sentence can have) given a theory of pruning. The function PossiblePrunings takes a proposition $p$, a set of alternatives $C$, and a constraint on pruning $con$, and returns all the pruning choices $C' \subseteq C$ such that $Exh(p,C')$ is licensed by $con$. 

In [8]:
jprint("Possible prunings and exhaustified meanings for $a \lor b$ given Pruning by Relevance: ")
[jprint("Pruning: ",i,"Meaning: ",exhf(Exh(a|b,i)))for i in PossiblePrunings(a|b,Alt,PrunRelevance)]

[None, None, None, None, None]

In [9]:
jprint("Possible prunings for $a \lor b$ given Pruning by Exhaustive Relevance: ")
[jprint("Pruning: ",i,"Meaning: ",exhf(Exh(a|b,i))) for i in PossiblePrunings(a|b,Alt,PrunExhaustiveRelevance)]

[None, None, None]

In [10]:
jprint("Possible prunings for $a \lor b$ given Pruning by Non-Settling: ")
[jprint("Pruning: ",i,"Meaning: ",exhf(Exh(a|b,i))) for i in PossiblePrunings(a|b,Alt,PrunSettling)]

[None, None, None]

We can see that in this case Exhaustive Relevance and Non-settling have the same (correct) predictions. With disjunction in the scope of a universal quantifier things change, and their predictions differ.

One can see in the following table (which lists all choices that are licit by at least one of these two constraints) that they are logically independent.

[Warning: takes a long time to compile (because of exhaustive relevance).]

In [11]:
# pos_prun_rel=PossiblePrunings(prej_universal,exh_universal.alts,PrunRelevance)
pos_prun_exhrel=PossiblePrunings(prej_universal,exh_universal.alts,PrunExhaustiveRelevance)
pos_prun_nonset=PossiblePrunings(prej_universal,exh_universal.alts,PrunSettling)

any_prun_universal=[i for i in ps(exh_universal.alts) if i in pos_prun_exhrel or i in pos_prun_nonset]
any_prun_universal_meaning=[exhf(Exh(prej_universal,i)) for i in any_prun_universal]

val_prun_exhrel=[i in pos_prun_exhrel for i in any_prun_universal]
val_prun_nonset=[i in pos_prun_nonset for i in any_prun_universal]

from prettytable import PrettyTable
t = PrettyTable()
t.add_column('Pruning choice',any_prun_universal)
t.add_column('Meaning',any_prun_universal_meaning)
t.add_column('Allowed by Exhaustive Relevance?',val_prun_exhrel)
t.add_column('Allowed by Non-Settling?',val_prun_nonset)
t._max_width = {"Pruning choice" : 25,"Meaning" : 25}
print(t)

+---------------------------+---------------------------+----------------------------------+--------------------------+
|       Pruning choice      |          Meaning          | Allowed by Exhaustive Relevance? | Allowed by Non-Settling? |
+---------------------------+---------------------------+----------------------------------+--------------------------+
|   [$\forall x, P(x) \lor  |   $\forall x, P(x) \lor   |               True               |          False           |
|           Q(x)$]          |           Q(x)$           |                                  |                          |
|   [$\forall x, P(x) \lor  |   $(\forall x, P(x) \lor  |               True               |          False           |
|  Q(x)$, $\forall x, P(x)  |  Q(x)) \land \neg[\forall |                                  |                          |
|        \land Q(x)$]       |    x, P(x) \land Q(x)]$   |                                  |                          |
|   [$\forall x, P(x) \lor  |   $(\foral

Exhaustive Relevance makes some obviously bad predictions. For instance, $[\forall x (Px \lor Qx), \forall x Px]$ is allowed. 

What about non-settling? It makes no obviously wrong predictions (some licit pruing choices look suspicious but are in fact harmless):

In [12]:
jprint("Possible prunings for $\forall x (Px \lor Qx)$ given Pruning by Non-Settling: ")
[jprint("Pruning: ",i,"Meaning: ",exhf(Exh(prej_universal,i))) for i in pos_prun_nonset]

[None, None, None, None, None, None, None, None, None]

We argue though that it is helpful to assume both Non-settling and Exhaustive Relevance. Assuming both, this is the set of licit prunings:

In [13]:
jprint("Possible prunings for $[\forall x (Px \lor Qx)]$ given both Pruning by Exhaustive relevance and Pruning by Non-Settling: ")
[jprint("Pruning: ",i,"Meaning: ",exhf(Exh(prej_universal,i))) for i in ps(exh_universal.alts) if i in pos_prun_nonset and i in pos_prun_exhrel]

[None, None, None, None, None]

The reason why we think it is helpful to have both constraints is that it will allow us to capture an otherwise intriguing data due to Denic (2020), showing that distributive inferences are impossible when the number of individuals in the domain of quantification equals the number of disjuncts. 

This is what we predict if we assume both constraints when the domain of quantification is 2:

In [14]:
from exh.model import options
options.dom_quant=2

prej_universal=Ax>P|Q
exh_universal=Exh(prej_universal)
# univ=Universe(fs=exh_universal.alts)

pos_prun_nonset_dom2=PossiblePrunings(prej_universal,exh_universal.alts,PrunSettling)
pos_prun_exhrel_nonset_dom2=[i for i in pos_prun_nonset_dom2 if PrunExhaustiveRelevance(prej_universal,exh_universal.alts,i)]

jprint("Possible prunings for $[\forall x (Px \lor Qx)]$ given both Pruning by Exhaustive relevance and Pruning by Non-Settling when the domain of quantification contains two individuals: ")
[jprint("Pruning: ",i,"Meaning: ",exhf(Exh(prej_universal,i))) for i in pos_prun_exhrel_nonset_dom2]

[None, None, None, None]

Note that now there is simply no way to get distributive inferences. Underlying reason: Once $\forall x Px$ is in the set of alternatives (which is ensured by Exhaustive Relevance whenever $\exists x Qx$ is in the set of alternatives),  then by (even simple) relevance $\exists x [Px \land Qx]$ can't be left out, because $\exists x (Px \land Qx)$ is equivalent to $(\forall x Px \land \exists x Qx) \lor (\forall x Qx \land \exists x Px) \lor (\exists x Px \land \exists x Qx \land \neg \forall x (Px \lor Qx))$, and the latter ends up in the union of cells.

In [15]:
diff=[i for i in pos_prun_nonset if i in pos_prun_exhrel if i not in pos_prun_exhrel_nonset_dom2]
jprint("Pruning choice that disappears when the domain of quantification contains 2 individuals: ")
[jprint("Pruning: ",i,"Meaning: ",exhf(Exh(prej_universal,i))) for i in diff]

[None]

What happens when the domain of quantification contains 3 individuals and there are three disjuncts?

Conjecture: $\exists x (Px \land Qx \land Rx)$ is unprunable, because equivalent to...:


In [25]:
# from exh.model import options
# options.dom_quant=3

# prej_universal3=Ax>P|Q|R
# exh_universal3=Exh(prej_universal3)

# univ=Universe(fs=exh_universal3.alts)
disjconj=(Ex>P&Q)|(Ex>P&R)|(Ex>Q&R)
# posrel=((Ax>P)&((Ex>Q)|(Ex>R)))|((Ax>Q)&((Ex>P)|(Ex>R)))|((Ax>R)&((Ex>P)|(Ex>Q)))|((Ex>~(P|Q|R))&(Ex>P)&(Ex>Q)&(Ex>R))
# jprint(posrel)
# univ.equivalent(disjconj,posrel)

In [4]:
from exh.model import options
options.dom_quant=3

prej_universal3=Ax>P|Q|R
universal3_alts=[prej_universal3,Ax>P&Q,Ax>P&R,Ax>Q&R,Ax>P,Ax>Q,Ax>R,
                Ex>P&Q,Ex>P&R,Ex>Q&R,Ex>P,Ex>Q,Ex>R,]
# universal3_alts_pruned=[prej_universal3,Ax>P&Q,Ax>P&R,Ax>Q&R,Ax>P,Ax>Q,Ax>R,
#                         Ex>P,Ex>Q,Ex>R,]

# PrunSettling(prej_universal3,universal3_alts,universal3_alts_pruned)
# PrunExhaustiveRelevance(prej_universal3,universal3_alts,universal3_alts_pruned)


# cells3=Cells(universal3_alts_pruned)
# len(cells3)
# jprint(exhf(exh_universal3))
pos_prun_nonset3=PossiblePrunings(prej_universal3,universal3_alts,PrunSettling)
# pos_prun_exhrel_nonset3=[i for i in pos_prun_nonset3 if PrunExhaustiveRelevance(prej_universal3,exh_universal3.alts,i)]


jprint("Possible prunings for $[\forall x (Px \lor Qx \lor Rx)]$ given both Pruning by Pruning by Non-Settling when the domain of quantification contains three individuals: ")
[jprint("Pruning: ",i,"Meaning: ",exhf(Exh(prej_universal3,i))) for i in pos_prun_nonset3]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [17]:
univ=Universe(fs=universal3_alts)
[i for i in pos_prun_nonset3 if univ.entails(Exh(prej_universal3,i),(Ex>P)&(Ex>Q)) and not univ.entails(Exh(prej_universal3,i),~(Ex>(P&Q)))]

[[$\forall x, P(x) \lor Q(x) \lor R(x)$,
  $\exists x, P(x)$,
  $\exists x, Q(x)$,
  $\exists x, R(x)$],
 [$\forall x, P(x) \lor Q(x) \lor R(x)$,
  $\forall x, P(x) \land Q(x)$,
  $\exists x, P(x)$,
  $\exists x, Q(x)$,
  $\exists x, R(x)$],
 [$\forall x, P(x) \lor Q(x) \lor R(x)$,
  $\forall x, P(x) \land R(x)$,
  $\exists x, P(x)$,
  $\exists x, Q(x)$,
  $\exists x, R(x)$],
 [$\forall x, P(x) \lor Q(x) \lor R(x)$,
  $\forall x, Q(x) \land R(x)$,
  $\exists x, P(x)$,
  $\exists x, Q(x)$,
  $\exists x, R(x)$],
 [$\forall x, P(x) \lor Q(x) \lor R(x)$,
  $\forall x, P(x) \land Q(x)$,
  $\forall x, P(x) \land R(x)$,
  $\exists x, P(x)$,
  $\exists x, Q(x)$,
  $\exists x, R(x)$],
 [$\forall x, P(x) \lor Q(x) \lor R(x)$,
  $\forall x, P(x) \land Q(x)$,
  $\forall x, Q(x) \land R(x)$,
  $\exists x, P(x)$,
  $\exists x, Q(x)$,
  $\exists x, R(x)$],
 [$\forall x, P(x) \lor Q(x) \lor R(x)$,
  $\forall x, P(x) \land R(x)$,
  $\forall x, Q(x) \land R(x)$,
  $\exists x, P(x)$,
  $\exists x, Q(x)$,

In [56]:
count=0
for i in cells3:
    jprint(count, i)
    count+=1

In [57]:
enorm=cells3[6]|cells3[8]|cells3[9]|cells3[13]|cells3[15]|cells3[17]|cells3[19]|cells3[20]|cells3[21]|cells3[23]|cells3[24]|cells3[25]|cells3[26]|cells3[27]|cells3[28]|cells3[29]|cells3[30]

In [59]:
univ=Universe(fs=universal3_alts)
univ.equivalent(enorm,disjconj)

False

In [20]:
pos_prun_exhrel_nonset3=[]
for i in pos_prun_nonset3:
    if PrunExhaustiveRelevance(prej_universal3,universal3_alts,i):
        jprint("Pruning: ",i,"Meaning: ",exhf(Exh(prej_universal3,i)))
        pos_prun_exhrel_nonset3.append(i)

KeyboardInterrupt: 